In [5]:
import numpy as np
import pandas as pd
import scipy.io

import sklearn.linear_model

import cmdstanpy
import arviz as az

import bebi103

import bokeh.io
import bokeh.plotting
bokeh.io.output_notebook()

Loading BokehJS ...

In [2]:
import cmdstanpy; cmdstanpy.install_cmdstan(compiler=True)

15:12:12 - cmdstanpy - INFO - Add C++ toolchain to $PATH: C:\Users\ailen\.cmdstan\RTools40


CmdStan install directory: C:\Users\ailen\.cmdstan
CmdStan version 2.36.0 already installed
Test model compilation


True

In [6]:
df = pd.read_csv("csv/all_subjects.csv")

# Aggregation
agg_df = df.groupby(['subj_id', 'n_flash', 'n_beep', 'blindspot', 'response_type', 'response']).size().reset_index(name='count')
pivot_df = agg_df.pivot_table(index=['subj_id', 'n_flash', 'n_beep', 'blindspot', 'response_type'],
                              columns='response', values='count', fill_value=0).reset_index()
pivot_df = pivot_df.rename(columns=lambda x: f'y{x}' if isinstance(x, int) else x)

subj_data = pivot_df[pivot_df['subj_id'] == "SV009"]
N = subj_data.shape[0]
data = {
    'N': N,
    'xA': subj_data['n_beep'].values,
    'xV': subj_data['n_flash'].values,
    'sA': subj_data['n_beep'].values,
    'sV': subj_data['n_flash'].values,
    'loc': subj_data['blindspot'].values,
    'strat': 1  # Strategy (1 = model averaging, 2 = model selection, 3 = probability matching)
}

with bebi103.stan.disable_logging():
    sm = cmdstanpy.CmdStanModel(stan_file="bci_with_strategy.stan")
    samples = az.from_cmdstanpy(sm.sample(data=data))

bebi103.stan.check_all_diagnostics(samples)

ValueError: Failed to get source info for Stan model 'C:\Users\ailen\Desktop\GitHub\blindspot-multisensory\Data\bci_with_strategy.stan'. Console:
Semantic error in 'C:\Users\ailen\Desktop\GitHub\blindspot-multisensory\Data\bci_with_strategy.stan', line 79, column 23 to column 47:
   -------------------------------------------------
    77:      p_common_sampled[n] = uniform_rng(0, 1);
    78:  
    79:      y_pred[n] = max(1, min(4, round(sV_hat[n])));
                                ^
    80:    }
    81:  }
   -------------------------------------------------

Ill-typed arguments supplied to function 'min':
(int, real)
Available signatures:
(int, int) => int
  The second argument must be int but got real
(array[] int) => int
  Expected 1 arguments but found 2 arguments.
(array[] real) => real
  Expected 1 arguments but found 2 arguments.
(vector) => real
  Expected 1 arguments but found 2 arguments.
(row_vector) => real
  Expected 1 arguments but found 2 arguments.
(Additional signat

In [3]:
import os, glob

import numpy as np

import bebi103
import cmdstanpy
import arviz as az

import bokeh.plotting
import bokeh.io
bokeh.io.output_notebook()

schools_data = {
    "J": 8,
    "y": [28, 8, -3, 7, -1, 1, 18, 12],
    "sigma": [15, 10, 16, 11, 9, 11, 10, 18],
}

schools_code = """
data {
  int<lower=0> J; // number of schools
  vector[J] y; // estimated treatment effects
  vector<lower=0>[J] sigma; // s.e. of effect estimates
}

parameters {
  real mu;
  real<lower=0> tau;
  vector[J] eta;
}

transformed parameters {
  vector[J] theta = mu + tau * eta;
}

model {
  eta ~ normal(0, 1);
  y ~ normal(theta, sigma);
}
"""

with open("schools_code.stan", "w") as f:
    f.write(schools_code)

with bebi103.stan.disable_logging():
    sm = cmdstanpy.CmdStanModel(stan_file="schools_code.stan")
    samples = sm.sample(data=schools_data, output_dir="./", show_progress=False)

samples = az.from_cmdstanpy(samples)

# Clean up
bebi103.stan.clean_cmdstan()
for fname in glob.glob("schools_code*"):
    os.remove(fname) 

# Make a plot of samples
p = bokeh.plotting.figure(
    frame_height=250, frame_width=250, x_axis_label="μ", y_axis_label="τ"
)
p.scatter(
    np.ravel(samples.posterior["mu"]), 
    np.ravel(samples.posterior["tau"]), 
    alpha=0.1
)

bokeh.io.show(p)

Loading BokehJS ...